In [2]:
import pandas as pd

In [3]:
df_rainforest = pd.read_csv('../RainforestAPI/rainforest_tvdeals_20220705.csv')
df_target = pd.read_csv('./target_tvdeals_20220703.csv')

In [4]:
target_specification = {
    "data_source":[],
    "title":[],
    "brand":[],
    "deal_price":[],
    "list_price":[],
    "multi_item_t_f":[],
    "avg_rating":[],
    "amount_rating":[],
    "deal_start":[],
    "deal_end":[],
    "description":[]
}

In [5]:
for index, row in df_rainforest.iterrows():
    target_specification["data_source"] += ["Rainforest API"]
    target_specification["title"] += [row["title"]]
    target_specification["brand"] += [row["merchant_name"]]
    try:
        target_specification["deal_price"] += [eval(row["deal_price"])["raw"]]
    except Exception:
        target_specification["deal_price"] += [None]
    try:
        target_specification["list_price"] += [eval(row["list_price"])["raw"]]
    except Exception:
        target_specification["list_price"] += [None]
    target_specification["multi_item_t_f"] += [str(row["type"]=="multi_item")]
    target_specification["avg_rating"] += [row["rating"]]
    target_specification["amount_rating"] += [row["ratings_total"]]
    target_specification["deal_start"] += [row["starts_at"]]
    target_specification["deal_end"] += [row["ends_at"]]
    target_specification["description"] += [row["description"]]

In [6]:
for index, row in df_target.iterrows():
    target_specification["data_source"] += ["Target Deals Website"]
    target_specification["title"] += [row["name"]]
    target_specification["brand"] += [row["brand"]]
    target_specification["deal_price"] += [row["deal_price"]]
    target_specification["list_price"] += [row["regular_price"]]
    target_specification["multi_item_t_f"] += [False]
    target_specification["avg_rating"] += [row["avg_overall_rating"]]
    target_specification["amount_rating"] += [row["amount_of_ratings"]]
    target_specification["deal_start"] += [None]
    target_specification["deal_end"] += [None]
    target_specification["description"] += [row["product_highlights"]]

In [78]:
df = pd.DataFrame.from_dict(target_specification)


In [79]:
# add column 'type' based on product name inputs
# types include: TV, TV Accessory, Mixed

def get_product_type(title):
    title = title.lower()
    tv_names = ["fire tv", "smart tv", "google tv", "nanocell tv", "tvs", "roku tv", "series", "uhd tv",
                "led tv", "hdr tv", "lcd tv", "hdtv", "outdoor tv"]
    accessory_names = ["stick", "player", "player", "mount", "movie screen", "projector screen",
                       "cleaner kit", "tv stand", "tv cube"]
    mixed_names = ["products", "accessories", "% discount from", "deals from", "supplies", "kit", "electronics"]
    if any([name in title for name in tv_names]) and not any([accessory in title for accessory in ["stick", "tv cube"]]):
        return "TV"
    elif any([name in title for name in accessory_names]):
        return "Accessory"
    elif any([name in title for name in mixed_names]):
        return "Mixed"
    else:
        return None

col_index = df.columns.get_loc("title") + 1
type_column = df["title"].apply(get_product_type)
df.insert(col_index, "type", type_column)

In [82]:
df.to_csv('merged_dataset2.csv', index=False)